<a href="https://colab.research.google.com/github/gcosma/personalised_mltc/blob/main/DECODECohortPaper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tool for analysing condition progression in patients with Multiple Long Term Conditions and Intellectual Disabilities

In [1]:
#@title Step 1: You need to make some installations. This is very simple. On the next line you will see a little triangle that's in a little black circle on the left of this box. Click on it. When the triangle turns to a green tick you will see a message that says "Done! Now click on Step 2".
# Install necessary packages
!pip -q install pandas numpy networkx matplotlib pyvis ipywidgets ipython

# Display a message after installation
print("Done! Now click on Step 2.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 35.1 MB/s eta 0:00:00
Done! Now click on Step 2.


In [2]:
#@title Step 2: The arrow on the left just before the start of this sentence is to hide and show the code. If you click it accidentally, just click it again and the code will be hidden. Below this sentence there is another arrow in a circle. To execute the code click on the arrow that's in the cirle.
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import colorsys
from pyvis.network import Network
import os
from google.colab import files
from IPython.display import HTML, display
import base64
import io
import colorsys
import ipywidgets as widgets
import math
import random
from datetime import datetime, timedelta
from itertools import combinations

# Global variables
global_data = None
total_patients_in_group = 0
gender = ''
age_group = ''
results_df = None

# 1. Replace the condition_categories dictionary with:
condition_categories = {
    "Anaemia": "Blood",
    "Cardiac Arrhythmias": "Circulatory",
    "Coronary Heart Disease": "Circulatory",
    "Heart Failure": "Circulatory",
    "Hypertension": "Circulatory",
    "Peripheral Vascular Disease": "Circulatory",
    "Stroke": "Circulatory",
    "Barretts Oesophagus": "Digestive",
    "Chronic Constipation": "Digestive",
    "Chronic Diarrhoea": "Digestive",
    "Cirrhosis": "Digestive",
    "Dysphagia": "Digestive",
    "Inflammatory Bowel Disease": "Digestive",
    "Reflux Disorders": "Digestive",
    "Hearing Loss": "Ear",
    "Addisons Disease": "Endocrine",
    "Diabetes": "Endocrine",
    "Polycystic Ovary Syndrome": "Endocrine",
    "Thyroid Disorders": "Endocrine",
    "Visual Impairment": "Eye",
    "Chronic Kidney Disease": "Genitourinary",
    "Menopausal And Perimenopausal": "Genitourinary",
    "Menopausal and Perimenopausal": "Genitourinary",
    "Dementia": "Mental",
    "Mental Illness": "Mental",
    "Tourette": "Mental",
    "Chronic Arthritis": "Musculoskeletal",
    "Chronic Pain Conditions": "Musculoskeletal",
    "Osteoporosis": "Musculoskeletal",
    "Cancer": "Neoplasms",
    "Cerebral Palsy": "Nervous",
    "Epilepsy": "Nervous",
    "Insomnia": "Nervous",
    "Multiple Sclerosis": "Nervous",
    "Neuropathic Pain": "Nervous",
    "Parkinsons": "Nervous",
    "Bronchiectasis": "Respiratory",
    "Chronic Airway Diseases": "Respiratory",
    "Chronic Pneumonia": "Respiratory",
    "Interstitial Lung Disease": "Respiratory",
    "Psoriasis": "Skin"
}

# 2. Add this SYSTEM_COLORS dictionary after the condition_categories:
SYSTEM_COLORS = {
    "Blood": "#DC143C",
    "Circulatory": "#FF4500",
    "Digestive": "#32CD32",
    "Ear": "#4169E1",
    "Endocrine": "#BA55D3",
    "Eye": "#20B2AA",
    "Genitourinary": "#DAA520",
    "Mental": "#8B4513",
    "Musculoskeletal": "#4682B4",
    "Neoplasms": "#800080",
    "Nervous": "#FFD700",
    "Respiratory": "#48D1CC",
    "Skin": "#F08080",
    "Other": "#808080"
}


# Generate color mapping
category_colors = None

def load_file():
    print("Please upload your CSV file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
        return None, None, None, None

    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    try:
        data = pd.read_csv(io.BytesIO(file_content))
        total_patients = data['TotalPatientsInGroup'].iloc[0]

        print(f"Uploaded file: {file_name}")

        # Normalize the filename for comparison
        file_name_lower = file_name.lower()

        # Check for gender
        if 'females' in file_name_lower:
            gender = 'Female'
        elif 'males' in file_name_lower:
            gender = 'Male'
        else:
            gender = 'Unknown Gender'

        # Check for age group
        if 'below45' in file_name_lower:
            age_group = '<45'
        elif '45to64' in file_name_lower:
            age_group = '45-64'
        elif '65plus' in file_name_lower:
            age_group = '65+'
        else:
            age_group = 'Unknown Age Group'

        print(f"File loaded successfully. Total patients: {total_patients}")
        print(f"Gender: {gender}, Age Group: {age_group}")

        return data, total_patients, gender, age_group

    except Exception as e:
        print(f"Error loading file: {str(e)}")
        return None, None, None, None

# 4. Replace the initialize_data function with:
def initialize_data(data, total_patients, gender_info, age_group_info):
    global global_data, total_patients_in_group, gender, age_group
    global_data = data
    total_patients_in_group = total_patients
    gender = gender_info
    age_group = age_group_info

def fixed_edge_color():
    return '#006400'  # Dark green color

def create_graph(data, min_percentage, min_frequency):
    """Create a NetworkX graph from the filtered data"""
    G = nx.DiGraph()

    for _, row in data.iterrows():
        if row['Percentage'] >= min_percentage and row['PairFrequency'] >= min_frequency:
            G.add_edge(
                row['ConditionA'],
                row['ConditionB'],
                weight=row['MedianDurationYears'],
                odds_ratio=row['OddsRatio'],
                pair_frequency=row['PairFrequency'],
                percentage=row['Percentage']
            )
    return G



def generate_caption(title=""):
    caption = f"""
    <div style="font-family: Arial, sans-serif; margin-bottom: 20px;">
        <h3 style="color: #333;">{title}</h3>
        <p style="color: #666; font-size: 14px;">
            Gender: {gender} | Age Group: {age_group} |
            Total Patients: {total_patients_in_group:,} |
            Condition Pairs: {len(global_data)}
        </p>
    </div>
    """
    return caption


def analyze_condition_combinations(min_percentage, min_frequency):
    global results_df

    if global_data is None or global_data.empty:
        print("No data available. Please load a valid CSV file.")
        return

    data = global_data[(global_data['Percentage'] >= min_percentage) &
                       (global_data['PairFrequency'] >= min_frequency)].copy()

    data.loc[:, 'ConditionA'] = data['ConditionA'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    data.loc[:, 'ConditionB'] = data['ConditionB'].str.replace(r'\s*\([^)]*\)', '', regex=True)
    data.loc[:, 'ConditionA'] = data['ConditionA'].str.replace('_', ' ')
    data.loc[:, 'ConditionB'] = data['ConditionB'].str.replace('_', ' ')

    unique_conditions = pd.unique(data[['ConditionA', 'ConditionB']].values.ravel('K'))

    pair_frequency_map = {}
    condition_frequency_map = {}

    for _, row in data.iterrows():
        key1 = f"{row['ConditionA']}_{row['ConditionB']}"
        key2 = f"{row['ConditionB']}_{row['ConditionA']}"
        pair_frequency_map[key1] = row['PairFrequency']
        pair_frequency_map[key2] = row['PairFrequency']

        condition_frequency_map[row['ConditionA']] = condition_frequency_map.get(row['ConditionA'], 0) + row['PairFrequency']
        condition_frequency_map[row['ConditionB']] = condition_frequency_map.get(row['ConditionB'], 0) + row['PairFrequency']

    result_data = []

    for k in range(3, min(8, len(unique_conditions) + 1)):
        for comb in combinations(unique_conditions, k):
            pair_frequencies = [pair_frequency_map.get(f"{a}_{b}", 0) for a, b in combinations(comb, 2)]
            frequency = min(pair_frequencies)
            prevalence = (frequency / total_patients_in_group) * 100

            observed = frequency
            expected = total_patients_in_group
            for condition in comb:
                expected *= (condition_frequency_map[condition] / total_patients_in_group)
            odds_ratio = observed / expected if expected != 0 else float('inf')

            result_data.append({
                'Combination': ' + '.join(comb),
                'NumConditions': len(comb),
                'Minimum Pair Frequency': frequency,
                'Prevalence of the combination (%)': prevalence,
                'Total odds ratio': odds_ratio
            })

    results_df = pd.DataFrame(result_data)
    results_df = results_df.sort_values('Prevalence of the combination (%)', ascending=False)
    results_df = results_df[results_df['Prevalence of the combination (%)'] > 0]

    print(f'Analysis complete. {len(results_df)} combinations found.')
    return results_df

def save_results_to_csv(filename="condition_combinations.csv"):
    global results_df
    if results_df is not None and not results_df.empty:
        results_df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")

        # For Colab, we need to provide a download link
        files.download(filename)
    else:
        print("No results available to save. Please run the analysis first.")

# 6. Update the display_legend function to use SYSTEM_COLORS:
def display_legend():
    print("\nSystem Legend:")
    unique_categories = sorted(set(condition_categories.values()))
    max_category_length = max(len(category) for category in unique_categories)

    for category in unique_categories:
        color = SYSTEM_COLORS.get(category, SYSTEM_COLORS["Other"])
        r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
        print(f"\033[48;2;{r};{g};{b}m  \033[0m \033[38;2;{r};{g};{b}m{category.ljust(max_category_length)}\033[0m")

def create_graph(data, min_percentage, min_frequency):
    """Create a NetworkX graph from the filtered data"""
    G = nx.DiGraph()

    for _, row in data.iterrows():
        if row['Percentage'] >= min_percentage and row['PairFrequency'] >= min_frequency:
            G.add_edge(
                row['ConditionA'],
                row['ConditionB'],
                weight=row['MedianDurationYears'],
                odds_ratio=row['OddsRatio'],
                pair_frequency=row['PairFrequency'],
                percentage=row['Percentage']
            )
    return G

def create_pyvis_network(G, height="800px"):
    """Create a PyVis network visualization from a NetworkX graph"""
    net = Network(notebook=True, bgcolor='white', font_color='black', cdn_resources='in_line', height=height, width="100%")

    # Add category legend
    legend_start_x = -600
    legend_start_y = -300
    net.add_node(
        "legend_title",
        label="Legend",
        x=legend_start_x,
        y=legend_start_y - 50,
        size=0,
        font={'size': 20, 'bold': True},
        physics=False,
        fixed=True
    )

    # Add system categories to legend
    unique_categories = sorted(set(condition_categories.values()))
    for i, category in enumerate(unique_categories):
        color = SYSTEM_COLORS[category]
        net.add_node(
            f"legend_{category}",
            label=f"{category}",
            x=legend_start_x + 50,
            y=legend_start_y + (i * 60),
            size=10,
            shape='dot',
            color={'background': color, 'border': color},
            font={'size': 16, 'align': 'left'},
            physics=False,
            fixed=True
        )

    # Calculate positions for nodes
    category_positions = {}
    radius = 300
    active_categories = set(condition_categories[node] for node in G.nodes if node in condition_categories)

    for i, category in enumerate(sorted(active_categories)):
        angle = i * (2 * math.pi / len(active_categories))
        category_positions[category] = {
            'x': radius * math.cos(angle),
            'y': radius * math.sin(angle)
        }

    # Add nodes
    for node in G.nodes:
        category = condition_categories.get(node, "Other")
        base_color = SYSTEM_COLORS.get(category, "#808080")
        pos = category_positions.get(category, {'x': 0, 'y': 0})

        x = pos['x'] + random.uniform(-50, 50)
        y = pos['y'] + random.uniform(-50, 50)

        net.add_node(
            node,
            label=node,
            title=f"{node}\nCategory: {category}",
            x=x,
            y=y,
            size=45,
            color={'background': f"{base_color}50", 'border': base_color},
            font={'size': 14, 'color': 'black'},
            physics=False
        )

    # Add edges with OR and median years
    for edge in G.edges(data=True):
        source, target = edge[0], edge[1]
        data = edge[2]

        # Create edge label showing OR and median years
        edge_label = f"OR: {data['odds_ratio']:.1f}\nYears: {data['weight']:.1f}"

        net.add_edge(
            source,
            target,
            label=edge_label,  # Add label directly on edge
            title=edge_label,  # Show in tooltip as well
            width=2,  # Fixed width for better readability
            color={'color': 'rgba(128,128,128,0.7)', 'highlight': 'black'},
            arrows={'to': {'enabled': True, 'scaleFactor': 0.5}},
            smooth={'type': 'curvedCW', 'roundness': 0.2},
            font={'size': 12, 'align': 'horizontal'}  # Style for edge label
        )

    # Network physics and interaction settings
    net.set_options("""
    {
        "physics": {
            "enabled": false
        },
        "edges": {
            "font": {
                "size": 12,
                "align": "horizontal",
                "background": "white"
            },
            "color": {"inherit": false},
            "smooth": {"type": "curvedCW", "roundness": 0.2}
        },
        "interaction": {
            "hover": true,
            "navigationButtons": true,
            "keyboard": true
        }
    }
    """)

    return net

def display_network(net, filename=None):
    """Display the network with improved layout and instructions"""
    if filename is None:
        filename = f"network_graph_{np.random.randint(0, 1000000)}.html"

    # Generate HTML content
    html = net.generate_html()

    with open(filename, 'w', encoding='utf-8') as f:
        f.write(html)

    with open(filename, 'r', encoding='utf-8') as f:
        html_content = f.read()

    b64 = base64.b64encode(html_content.encode()).decode()

    # Display the graph with instructions panel
    display(HTML(f"""
    <div style="display: flex; justify-content: space-between;">
        <div style="width: 25%;">
            <h3>How to Read the Graph:</h3>
            <ul>
                <li>Nodes represent conditions, colored by body system category.</li>
                <li>Edge labels show:
                    <ul>
                        <li>OR: Odds Ratio</li>
                        <li>Years: Median progression time</li>
                    </ul>
                </li>
                <li>Edge direction shows typical progression</li>
            </ul>
            <h4>Interactions:</h4>
            <ul>
                <li>Hover over nodes and edges for details</li>
                <li>Drag nodes to adjust layout</li>
                <li>Scroll to zoom in/out</li>
            </ul>
        </div>
        <div style="width: 75%;">
            <iframe src="data:text/html;base64,{b64}" width="100%" height="800px"></iframe>
        </div>
    </div>
    """))

    # Add download button
    save_button = widgets.Button(description="Save Graph as HTML")
    save_output = widgets.Output()

    def on_save_button_clicked(b):
        with save_output:
            save_output.clear_output()
            files.download(filename)
            print(f"Graph saved as {filename}")

    save_button.on_click(on_save_button_clicked)
    display(save_button, save_output)

def create_timeline_visualization(patient_conditions, diagnosis_dates, data, min_or=2.0):
    """
    Create an interactive timeline visualization showing patient trajectory against cohort patterns

    Args:
        patient_conditions: List of tuples [(condition, date), ...] with patient's diagnoses and dates
        diagnosis_dates: Dictionary of {condition: datetime} for patient's diagnoses
        data: DataFrame with cohort trajectory data
        min_or: Minimum odds ratio to consider for cohort patterns
    """
    # Sort patient's conditions by date
    patient_timeline = sorted([(date, cond) for cond, date in diagnosis_dates.items()])

    # Create figure with secondary y-axis
    fig = go.Figure()

    # Add patient's actual trajectory
    patient_x = [date for date, _ in patient_timeline]
    patient_y = list(range(1, len(patient_timeline) + 1))
    patient_text = [cond for _, cond in patient_timeline]

    fig.add_trace(
        go.Scatter(
            x=patient_x,
            y=patient_y,
            mode='lines+markers+text',
            name="Patient's Trajectory",
            line=dict(color='red', width=3),
            marker=dict(size=12),
            text=patient_text,
            textposition="top center",
            hovertemplate=(
                "<b>%{text}</b><br>" +
                "Date: %{x}<br>" +
                "<extra></extra>"
            )
        )
    )

    # Add cohort patterns
    filtered_data = data[data['OddsRatio'] >= min_or]

    # For each patient condition, add potential subsequent conditions
    for i, (date, condition) in enumerate(patient_timeline):
        if i < len(patient_timeline) - 1:
            next_date = patient_timeline[i + 1][0]

            # Find common next conditions in cohort
            cohort_trajectories = filtered_data[
                filtered_data['ConditionA'] == condition
            ]

            for _, trajectory in cohort_trajectories.iterrows():
                # Parse median time
                median_str = trajectory['MedianDurationYearsWithIQR'].split(' ')[0]
                median_years = float(median_str)

                # Calculate expected date
                expected_date = date + timedelta(days=median_years*365)

                if expected_date <= next_date:  # Only show patterns within actual timeline
                    fig.add_trace(
                        go.Scatter(
                            x=[date, expected_date],
                            y=[i+1, i+2],
                            mode='lines',
                            name=f'Cohort Pattern: {trajectory["ConditionB"]}',
                            line=dict(
                                color='rgba(100,100,100,0.2)',
                                dash='dot'
                            ),
                            hovertemplate=(
                                f"<b>{trajectory['ConditionB']}</b><br>" +
                                f"Following: {condition}<br>" +
                                f"OR: {trajectory['OddsRatio']:.1f}<br>" +
                                f"Expected: %{x}<br>" +
                                "<extra></extra>"
                            )
                        )
                    )

    # Update layout
    fig.update_layout(
        title="Patient Trajectory vs Cohort Patterns",
        xaxis_title="Date",
        yaxis_title="Cumulative Conditions",
        showlegend=True,
        hovermode='closest',
        height=600
    )

    return fig

def create_interactive_timeline():
    """Create interactive interface for timeline visualization"""
    condition_input = widgets.Text(
        value='',
        placeholder='Enter condition',
        description='Condition:',
        disabled=False
    )

    date_input = widgets.DatePicker(
        description='Date:',
        disabled=False
    )

    conditions_list = widgets.Output()
    patient_conditions = {}

    def add_condition(b):
        if condition_input.value and date_input.value:
            patient_conditions[condition_input.value] = date_input.value
            with conditions_list:
                conditions_list.clear_output()
                print("Current conditions:")
                for cond, date in sorted(patient_conditions.items(), key=lambda x: x[1]):
                    print(f"{date.strftime('%Y-%m-%d')}: {cond}")

    add_button = widgets.Button(description="Add Condition")
    add_button.on_click(add_condition)

    min_or_slider = widgets.FloatSlider(
        value=2.0,
        min=1.0,
        max=10.0,
        step=0.5,
        description='Min OR:',
        continuous_update=False
    )

    plot_output = widgets.Output()

    def update_plot(b):
        with plot_output:
            plot_output.clear_output()
            if patient_conditions:
                fig = create_timeline_visualization(
                    list(patient_conditions.items()),
                    patient_conditions,
                    global_data,
                    min_or_slider.value
                )
                fig.show()

    update_button = widgets.Button(description="Update Visualization")
    update_button.on_click(update_plot)

    # Layout
    input_box = widgets.VBox([
        widgets.HBox([condition_input, date_input, add_button]),
        conditions_list,
        min_or_slider,
        update_button
    ])

    display(input_box, plot_output)

# Example usage
def run_example():
    # Sample patient data
    patient_conditions = {
        "Hypertension": datetime(2020, 1, 1),
        "Diabetes": datetime(2021, 3, 15),
        "Chronic Kidney Disease": datetime(2022, 8, 30)
    }

    fig = create_timeline_visualization(
        list(patient_conditions.items()),
        patient_conditions,
        global_data,
        min_or=2.0
    )
    fig.show()



# Main execution block
if __name__ == "__main__":
    try:
        # Load the file
        data, total_patients, gender, age_group = load_file()

        if data is not None:
            # Initialize the data
            initialize_data(data, total_patients, gender, age_group)

            # Display the legend once at the beginning
            display_legend()

            while True:
                # Calculate ranges
                min_freq_range = (global_data['PairFrequency'].min(), global_data['PairFrequency'].max())
                min_percentage_range = (global_data['Percentage'].min(), global_data['Percentage'].max())

                # Get user input for minimum pair frequency and percentage with validation
                while True:
                    try:
                        min_frequency = int(input(f"Enter the minimum pair frequency [{min_freq_range[0]}-{min_freq_range[1]}]: "))
                        if min_freq_range[0] <= min_frequency <= min_freq_range[1]:
                            break
                        else:
                            print(f"Error: Value must be between {min_freq_range[0]} and {min_freq_range[1]}. Please try again.")
                    except ValueError:
                        print("Error: Please enter a valid integer.")

                while True:
                    try:
                        min_percentage = float(input(f"Enter the minimum prevalance percentage of a pair (%) [{min_percentage_range[0]:.2f}-{min_percentage_range[1]:.2f}]: "))
                        if min_percentage_range[0] <= min_percentage <= min_percentage_range[1]:
                            break
                        else:
                            print(f"Error: Value must be between {min_percentage_range[0]:.2f} and {min_percentage_range[1]:.2f}. Please try again.")
                    except ValueError:
                        print("Error: Please enter a valid number.")

                # Update and display the graph
                update_graph(min_percentage, min_frequency)

                # Analyze condition combinations
                results = analyze_condition_combinations(min_percentage, min_frequency)

                # Display total number of condition combinations and all combinations
                total_combinations = len(results)
                print(f"\nTotal number of condition combinations: {total_combinations}")
                print("All condition combinations:")

                # Display all results without truncation
                pd.set_option('display.max_rows', None)
                pd.set_option('display.max_columns', None)
                pd.set_option('display.width', None)
                pd.set_option('display.max_colwidth', None)
                display(results)

                # Reset display options to default
                pd.reset_option('display.max_rows')
                pd.reset_option('display.max_columns')
                pd.reset_option('display.width')
                pd.reset_option('display.max_colwidth')

                # Ask if user wants to save results
                save_choice = input("Do you want to save the results to a CSV file? (yes/no): ").lower()
                if save_choice == 'yes':
                    save_results_to_csv()

                # Ask if the user wants to run again with different parameters
                run_again = input("Do you want to run the analysis again with different parameters? (yes/no): ").lower()
                if run_again != 'yes':
                    break

            print("Analysis completed. Thank you for using the script!")
        else:
            print("Failed to load data. Please run the script again and upload a valid CSV file.")

    except KeyboardInterrupt:
        print("\nScript execution interrupted by user. Exiting gracefully...")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
    finally:
        print("Script execution completed.")

print("You can run the cell again to start a new analysis with a different file.")

Please upload your CSV file.


Saving Females_below45.csv to Females_below45.csv
Uploaded file: Females_below45.csv
File loaded successfully. Total patients: 6397
Gender: Female, Age Group: <45

System Legend:
   Blood          
   Circulatory    
   Digestive      
   Ear            
   Endocrine      
   Eye            
   Genitourinary  
   Mental         
   Musculoskeletal
   Neoplasms      
   Nervous        
   Respiratory    
   Skin           
Enter the minimum pair frequency [71-484]: 100
Enter the minimum prevalance percentage of a pair (%) [1.11-7.57]: 2
An unexpected error occurred: name 'update_graph' is not defined
Script execution completed.
You can run the cell again to start a new analysis with a different file.


In [3]:
#@title Step 2: The arrow on the left just before the start of this sentence is to hide and show the code. If you click it accidentally, just click it again and the code will be hidden. Below this sentence there is another arrow in a circle. To execute the code click on the arrow that's in the cirle.
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import colorsys
from pyvis.network import Network
import os
from google.colab import files
from IPython.display import HTML, display
import base64
import io
import colorsys
import ipywidgets as widgets
import math
import random
from datetime import datetime, timedelta
from itertools import combinations

# Global variables
global_data = None
total_patients_in_group = 0
gender = ''
age_group = ''
results_df = None

# 1. Replace the condition_categories dictionary with:
condition_categories = {
    "Anaemia": "Blood",
    "Cardiac Arrhythmias": "Circulatory",
    "Coronary Heart Disease": "Circulatory",
    "Heart Failure": "Circulatory",
    "Hypertension": "Circulatory",
    "Peripheral Vascular Disease": "Circulatory",
    "Stroke": "Circulatory",
    "Barretts Oesophagus": "Digestive",
    "Chronic Constipation": "Digestive",
    "Chronic Diarrhoea": "Digestive",
    "Cirrhosis": "Digestive",
    "Dysphagia": "Digestive",
    "Inflammatory Bowel Disease": "Digestive",
    "Reflux Disorders": "Digestive",
    "Hearing Loss": "Ear",
    "Addisons Disease": "Endocrine",
    "Diabetes": "Endocrine",
    "Polycystic Ovary Syndrome": "Endocrine",
    "Thyroid Disorders": "Endocrine",
    "Visual Impairment": "Eye",
    "Chronic Kidney Disease": "Genitourinary",
    "Menopausal And Perimenopausal": "Genitourinary",
    "Menopausal and Perimenopausal": "Genitourinary",
    "Dementia": "Mental",
    "Mental Illness": "Mental",
    "Tourette": "Mental",
    "Chronic Arthritis": "Musculoskeletal",
    "Chronic Pain Conditions": "Musculoskeletal",
    "Osteoporosis": "Musculoskeletal",
    "Cancer": "Neoplasms",
    "Cerebral Palsy": "Nervous",
    "Epilepsy": "Nervous",
    "Insomnia": "Nervous",
    "Multiple Sclerosis": "Nervous",
    "Neuropathic Pain": "Nervous",
    "Parkinsons": "Nervous",
    "Bronchiectasis": "Respiratory",
    "Chronic Airway Diseases": "Respiratory",
    "Chronic Pneumonia": "Respiratory",
    "Interstitial Lung Disease": "Respiratory",
    "Psoriasis": "Skin"
}

# 2. Add this SYSTEM_COLORS dictionary after the condition_categories:
SYSTEM_COLORS = {
    "Blood": "#DC143C",        # Crimson
    "Circulatory": "#FF4500",  # Orange Red
    "Digestive": "#32CD32",    # Lime Green
    "Ear": "#4169E1",         # Royal Blue
    "Endocrine": "#BA55D3",    # Medium Orchid
    "Eye": "#1E90FF",         # Dodger Blue (changed from teal)
    "Genitourinary": "#DAA520", # Goldenrod
    "Mental": "#8B4513",       # Saddle Brown
    "Musculoskeletal": "#4682B4", # Steel Blue
    "Neoplasms": "#800080",    # Purple
    "Nervous": "#FFD700",      # Gold
    "Respiratory": "#006400",   # Dark Green (changed from teal)
    "Skin": "#F08080",        # Light Coral
    "Other": "#808080"         # Gray
}


# Generate color mapping
category_colors = None

def load_file():
    print("Please upload your CSV file.")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Exiting.")
        return None, None, None, None

    file_name = next(iter(uploaded))
    file_content = uploaded[file_name]

    try:
        data = pd.read_csv(io.BytesIO(file_content))
        total_patients = data['TotalPatientsInGroup'].iloc[0]

        print(f"Uploaded file: {file_name}")

        # Normalize the filename for comparison
        file_name_lower = file_name.lower()

        # Check for gender
        if 'females' in file_name_lower:
            gender = 'Female'
        elif 'males' in file_name_lower:
            gender = 'Male'
        else:
            gender = 'Unknown Gender'

        # Check for age group
        if 'below45' in file_name_lower:
            age_group = '<45'
        elif '45to64' in file_name_lower:
            age_group = '45-64'
        elif '65plus' in file_name_lower:
            age_group = '65+'
        else:
            age_group = 'Unknown Age Group'

        print(f"File loaded successfully. Total patients: {total_patients}")
        print(f"Gender: {gender}, Age Group: {age_group}")

        return data, total_patients, gender, age_group

    except Exception as e:
        print(f"Error loading file: {str(e)}")
        return None, None, None, None

def initialize_data(data, total_patients, gender_info, age_group_info):
    """Initialize global variables"""
    global global_data, total_patients_in_group, gender, age_group
    global_data = data
    total_patients_in_group = total_patients
    gender = gender_info
    age_group = age_group_info


def display_legend():
    """Display color-coded legend for body systems"""
    print("\nSystem Legend:")
    unique_categories = sorted(set(condition_categories.values()))
    max_category_length = max(len(category) for category in unique_categories)

    for category in unique_categories:
        color = SYSTEM_COLORS.get(category, SYSTEM_COLORS["Other"])
        r, g, b = int(color[1:3], 16), int(color[3:5], 16), int(color[5:7], 16)
        print(f"\033[48;2;{r};{g};{b}m  \033[0m \033[38;2;{r};{g};{b}m{category.ljust(max_category_length)}\033[0m")

def validate_input_range(value, min_val, max_val, param_name):
    """Validate that input is within acceptable range"""
    if not min_val <= value <= max_val:
        raise ValueError(f"{param_name} must be between {min_val} and {max_val}")
    return value

def calculate_summary_stats(data, min_odds_ratio, min_frequency):
    """Calculate summary statistics for the filtered data"""
    filtered_data = data[
        (data['OddsRatio'] >= min_odds_ratio) &
        (data['PairFrequency'] >= min_frequency)
    ]

    stats = {
        'sex': gender,
        'age_group': age_group,
        'total_patients': total_patients_in_group,
        'odds_ratio_min': filtered_data['OddsRatio'].min(),
        'odds_ratio_max': filtered_data['OddsRatio'].max(),
        'min_prevalence': filtered_data['Percentage'].min(),
        'min_prevalence_patients': int((filtered_data['Percentage'].min() * total_patients_in_group) / 100),
        'condition_pairs': len(filtered_data),
        'total_odds_ratio': filtered_data['OddsRatio'].sum()
    }

    return stats

def generate_caption(title="", stats=None):
    """Generate HTML caption for the visualization with detailed statistics"""
    if stats is None:
        return ""

    caption = f"""
    <div style="font-family: Arial, sans-serif; margin-bottom: 20px;">
        <h3 style="color: #333;">{title}</h3>
        <p style="color: #666; font-size: 14px; line-height: 1.6;">
            <strong>Sex:</strong> {stats['sex']}.
            <strong>Age group:</strong> {stats['age_group']}.
            <strong>Total patients with diagnoses in this group:</strong> {stats['total_patients']:,}.
            <strong>Odds ratio range:</strong> [{stats['odds_ratio_min']:.2f} - {stats['odds_ratio_max']:.2f}].<br>
            <strong>Observed minimum prevalence:</strong> {stats['min_prevalence']:.2f}% ({stats['min_prevalence_patients']:,} patients).
            <strong>Number of condition pairs shown:</strong> {stats['condition_pairs']}.
            <strong>Total sum of odds ratios:</strong> {stats['total_odds_ratio']:.2f}.
        </p>
    </div>
    """
    return caption

def clean_condition_names(data):
    """Clean condition names by removing parentheses and underscores"""
    data = data.copy()
    data['ConditionA'] = data['ConditionA'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.replace('_', ' ')
    data['ConditionB'] = data['ConditionB'].str.replace(r'\s*\([^)]*\)', '', regex=True).str.replace('_', ' ')
    return data

def create_filtered_graph(data, min_odds_ratio, min_frequency):
    """Create a filtered NetworkX graph"""
    # Filter the data based on odds ratio and frequency
    filtered_data = data[
        (data['OddsRatio'] >= min_odds_ratio) &
        (data['PairFrequency'] >= min_frequency)
    ].copy()

    # Clean the condition names
    filtered_data = clean_condition_names(filtered_data)

    # Create the directed graph
    G = nx.DiGraph()

    # Add edges with attributes
    for _, row in filtered_data.iterrows():
        G.add_edge(
            row['ConditionA'],
            row['ConditionB'],
            weight=row['MedianDurationYears'],
            odds_ratio=row['OddsRatio'],
            pair_frequency=row['PairFrequency'],
            percentage=row['Percentage']
        )

    return G


def create_condition_category_legend():
    """Create the condition categories legend HTML that will be embedded in the visualization"""
    legend_items = []
    for category, color in sorted(SYSTEM_COLORS.items()):
        if category != "Other":  # Only show actual categories, not the "Other" fallback
            legend_items.append(f"""
                <div style='margin: 2px 0;'>
                    <span style='display: inline-block; width: 20px; height: 20px; background-color: {color};
                          border: 1px solid {color}; margin-right: 5px; vertical-align: middle;'></span>
                    <span style='vertical-align: middle;'>{category}</span>
                </div>
            """)

    legend_html = f"""
        <div style='position: absolute; top: 10px; left: 10px; background: white; padding: 10px;
                    border: 1px solid #ccc; font-size: 12px; max-width: 200px;'>
            <div style='font-weight: bold; margin-bottom: 5px;'>Condition Categories</div>
            {''.join(legend_items)}
        </div>
    """
    return legend_html



def create_patient_count_legend(G):
    """Create a dynamic patient count legend based on actual data values"""
    # Extract pair frequencies from graph edges
    pair_frequencies = [data['pair_frequency'] for _, _, data in G.edges(data=True)]

    if len(pair_frequencies) == 0:
        return """<div>No data available for legend</div>"""

    # Calculate percentiles for the ranges
    percentiles = np.percentile(pair_frequencies, [0, 20, 40, 60, 80, 100])

    legend_html = """
        <div style='position: absolute; top: 10px; right: 10px; background: white; padding: 10px;
             border: 1px solid #ccc; font-size: 12px; z-index: 1000; box-shadow: 2px 2px 4px rgba(0,0,0,0.1);'>
            <div style='font-weight: bold; margin-bottom: 5px;'>Patient Count Ranges</div>
    """

    # Add legend items
    for i in range(4):
        legend_html += f"""
            <div style='margin: 5px 0;'>
                <div style='border-bottom: {i+1}px solid black; width: 40px; display: inline-block; margin-right: 5px;'></div>
                {percentiles[i]:.0f} ≤ Patients < {percentiles[i+1]:.0f} ({i*20}% - {(i+1)*20}%)</div>
        """

    # Add the final range
    legend_html += f"""
        <div style='margin: 5px 0;'>
            <div style='border-bottom: 5px solid black; width: 40px; display: inline-block; margin-right: 5px;'></div>
            Patients ≥ {percentiles[4]:.0f} (80%+)</div>
        </div>
    """

    return legend_html


def display_network(net, title="Condition Network", stats=None):
    """Display the network with a title and download option"""
    # Generate unique filename
    filename = f"network_graph_{random.randint(0, 1000000)}.html"

    # Save and read the HTML
    net.save_graph(filename)
    with open(filename, 'r', encoding='utf-8') as f:
        html_content = f.read()

    # Create base64 encoded version
    b64 = base64.b64encode(html_content.encode()).decode()

    # Generate caption with statistics
    caption = generate_caption(title, stats)

    # Display with title and instructions
    display(HTML(f"""
    {caption}
    <div style="display: flex; justify-content: space-between; position: relative;">
        <div style="width: 20%; padding: 10px;">
            <h3>How to Use:</h3>
            <ul>
                <li>Hover over nodes for condition details</li>
                <li>Hover over edges for relationship details</li>
                <li>Use mouse wheel to zoom</li>
                <li>Click and drag to pan</li>
            </ul>
        </div>
        <div style="width: 80%; position: relative;">
            <iframe src="data:text/html;base64,{b64}" width="100%" height="800px"></iframe>
        </div>
    </div>
    """))

    # Add download button
    download_button = widgets.Button(description="Download Graph")
    download_output = widgets.Output()

    def on_download_clicked(b):
        with download_output:
            download_output.clear_output()
            files.download(filename)

    download_button.on_click(on_download_clicked)
    display(download_button, download_output)

def display_legends(G):
    """Display legends separately from the network visualization"""

    # Create patient count legend
    pair_frequencies = [data['pair_frequency'] for _, _, data in G.edges(data=True)]
    if len(pair_frequencies) > 0:
        percentiles = np.percentile(pair_frequencies, [0, 20, 40, 60, 80, 100])

        # Patient count legend HTML
        patient_legend = """
        <div style='padding: 10px; margin: 10px; border: 1px solid #ccc; display: inline-block; background: white;'>
            <h3 style='margin-top: 0;'>Patient Count Ranges</h3>
        """

        # Add ranges
        for i in range(4):
            patient_legend += f"""
                <div style='margin: 5px 0;'>
                    <div style='border-bottom: {i+1}px solid black; width: 40px; display: inline-block; margin-right: 5px;'></div>
                    {percentiles[i]:.0f} ≤ Patients < {percentiles[i+1]:.0f} ({i*20}% - {(i+1)*20}%)</div>
            """

        # Add final range
        patient_legend += f"""
            <div style='margin: 5px 0;'>
                <div style='border-bottom: 5px solid black; width: 40px; display: inline-block; margin-right: 5px;'></div>
                Patients ≥ {percentiles[4]:.0f} (80%+)</div>
            </div>
        """

        # Category legend HTML
        category_legend = """
        <div style='padding: 10px; margin: 10px; border: 1px solid #ccc; display: inline-block; background: white;'>
            <h3 style='margin-top: 0;'>Condition Categories</h3>
        """

        # Add categories
        for category, color in sorted(SYSTEM_COLORS.items()):
            if category != "Other":
                category_legend += f"""
                    <div style='margin: 5px 0;'>
                        <span style='display: inline-block; width: 20px; height: 20px; background-color: {color};
                              border: 1px solid {color}; margin-right: 5px; vertical-align: middle;'></span>
                        <span style='vertical-align: middle;'>{category}</span>
                    </div>
                """
        category_legend += "</div>"

        # Display both legends
        display(HTML(f"""
        <div style='display: flex; flex-wrap: wrap; gap: 20px;'>
            {patient_legend}
            {category_legend}
        </div>
        """))
    else:
        print("No data available for legends")

def create_visualization(G, height="800px"):
    """Create a PyVis network visualization (without legends)"""
    net = Network(notebook=True, bgcolor='white', font_color='black', cdn_resources='in_line', height=height, width="100%")

    # Calculate positions for nodes based on categories
    category_positions = {}
    radius = 300
    active_categories = set(condition_categories[node] for node in G.nodes if node in condition_categories)

    for i, category in enumerate(sorted(active_categories)):
        angle = i * (2 * math.pi / len(active_categories))
        category_positions[category] = {
            'x': radius * math.cos(angle),
            'y': radius * math.sin(angle)
        }

    # Add nodes
    for node in G.nodes:
        category = condition_categories.get(node, "Other")
        base_color = SYSTEM_COLORS.get(category, "#808080")
        pos = category_positions.get(category, {'x': 0, 'y': 0})

        x = pos['x'] + random.uniform(-50, 50)
        y = pos['y'] + random.uniform(-50, 50)

        net.add_node(
            node,
            label=node,
            title=f"{node}\nCategory: {category}",
            x=x,
            y=y,
            size=45,
            color={'background': f"{base_color}50", 'border': base_color},
            font={'size': 14, 'color': 'black'},
            physics=False
        )

    # Get pair frequencies for edge widths
    pair_frequencies = [data['pair_frequency'] for _, _, data in G.edges(data=True)]
    if pair_frequencies:
        percentiles = np.percentile(pair_frequencies, [0, 20, 40, 60, 80, 100])
    else:
        percentiles = [0, 0, 0, 0, 0, 0]

    # Add edges
    for edge in G.edges(data=True):
        source, target = edge[0], edge[1]
        data = edge[2]

        freq = data['pair_frequency']
        if freq < percentiles[1]:
            width = 1
        elif freq < percentiles[2]:
            width = 2
        elif freq < percentiles[3]:
            width = 3
        elif freq < percentiles[4]:
            width = 4
        else:
            width = 5

        edge_label = f"OR: {data['odds_ratio']:.1f}\nYears: {data['weight']:.1f}"

        net.add_edge(
            source,
            target,
            label=edge_label,
            title=edge_label,
            width=width,
            color={'color': 'rgba(128,128,128,0.7)', 'highlight': 'black'},
            arrows={'to': {'enabled': True, 'scaleFactor': 0.5}},
            smooth={'type': 'curvedCW', 'roundness': 0.2},
            font={'size': 12, 'align': 'horizontal'}
        )

    # Set visualization options
    net.set_options("""
    {
        "physics": {
            "enabled": false
        },
        "edges": {
            "font": {
                "size": 12,
                "align": "horizontal",
                "background": "white"
            },
            "color": {"inherit": false},
            "smooth": {"type": "curvedCW", "roundness": 0.2}
        },
        "interaction": {
            "hover": true,
            "navigationButtons": true,
            "keyboard": true
        }
    }
    """)

    return net

def visualize_network(min_odds_ratio, min_frequency):
    """Main function to create and display the filtered network with separate legends"""
    if global_data is None:
        print("Please load data first using load_file()")
        return

    # Create filtered graph
    G = create_filtered_graph(global_data, min_odds_ratio, min_frequency)

    if len(G.nodes) == 0:
        print("No nodes match the filtering criteria. Please adjust the thresholds.")
        return

    # Calculate summary statistics
    stats = calculate_summary_stats(global_data, min_odds_ratio, min_frequency)

    # Display legends first
    display_legends(G)

    # Create and display visualization
    net = create_visualization(G)
    display_network(net, f"Filtered Condition Network (min OR: {min_odds_ratio}, min frequency: {min_frequency})", stats)


def update_graph(min_odds_ratio, min_frequency):
    """Update the graph with new filtering criteria"""
    G = create_filtered_graph(global_data, min_odds_ratio, min_frequency)
    if len(G.nodes) == 0:
        print("Warning: No nodes match the current filtering criteria.")
        return None
    net = create_visualization(G)
    return net

def analyze_condition_combinations(min_odds_ratio, min_frequency):
    """Analyze combinations of conditions based on odds ratio and frequency thresholds"""
    if global_data is None or global_data.empty:
        print("No data available. Please load a valid CSV file first.")
        return None

    filtered_data = global_data[
        (global_data['OddsRatio'] >= min_odds_ratio) &
        (global_data['PairFrequency'] >= min_frequency)
    ].copy()

    # Clean condition names
    filtered_data = clean_condition_names(filtered_data)

    # Get unique conditions
    unique_conditions = pd.unique(filtered_data[['ConditionA', 'ConditionB']].values.ravel('K'))

    # Create frequency maps
    pair_frequency_map = {}
    condition_frequency_map = {}

    for _, row in filtered_data.iterrows():
        # Map both directions of pair frequencies
        key1 = f"{row['ConditionA']}_{row['ConditionB']}"
        key2 = f"{row['ConditionB']}_{row['ConditionA']}"
        pair_frequency_map[key1] = row['PairFrequency']
        pair_frequency_map[key2]= row['PairFrequency']

        # Update individual condition frequencies
        condition_frequency_map[row['ConditionA']] = condition_frequency_map.get(row['ConditionA'], 0) + row['PairFrequency']
        condition_frequency_map[row['ConditionB']] = condition_frequency_map.get(row['ConditionB'], 0) + row['PairFrequency']

    return pd.DataFrame({
        'Conditions': list(unique_conditions),
        'Frequency': [condition_frequency_map.get(c, 0) for c in unique_conditions],
        'Prevalence': [condition_frequency_map.get(c, 0) / total_patients_in_group * 100 for c in unique_conditions]
    }).sort_values('Prevalence', ascending=False)

if __name__ == "__main__":
    try:
        data, total_patients, gender, age_group = load_file()

        if data is not None:
            initialize_data(data, total_patients, gender, age_group)
            display_legend()

            odds_ratio_range = (data['OddsRatio'].min(), data['OddsRatio'].max())
            frequency_range = (data['PairFrequency'].min(), data['PairFrequency'].max())

            while True:
                try:
                    min_odds_ratio = float(input(f"Enter minimum odds ratio [{odds_ratio_range[0]:.2f}-{odds_ratio_range[1]:.2f}]: "))
                    validate_input_range(min_odds_ratio, odds_ratio_range[0], odds_ratio_range[1], "Minimum odds ratio")
                    break
                except ValueError as e:
                    print(f"Error: {str(e)}")

            while True:
                try:
                    min_frequency = float(input(f"Enter minimum frequency [{frequency_range[0]:.0f}-{frequency_range[1]:.0f}]: "))
                    validate_input_range(min_frequency, frequency_range[0], frequency_range[1], "Minimum frequency")
                    break
                except ValueError as e:
                    print(f"Error: {str(e)}")

            visualize_network(min_odds_ratio, min_frequency)
            print("\nVisualization complete!")

    except KeyboardInterrupt:
        print("\nScript execution interrupted by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
    finally:
        print("\nScript execution completed.")



Please upload your CSV file.


Saving Females_below45.csv to Females_below45 (1).csv
Uploaded file: Females_below45 (1).csv
File loaded successfully. Total patients: 6397
Gender: Female, Age Group: <45

System Legend:
   Blood          
   Circulatory    
   Digestive      
   Ear            
   Endocrine      
   Eye            
   Genitourinary  
   Mental         
   Musculoskeletal
   Neoplasms      
   Nervous        
   Respiratory    
   Skin           
Enter minimum odds ratio [2.39-15.26]: 3
Enter minimum frequency [71-484]: 100


Button(description='Download Graph', style=ButtonStyle())

Output()


Visualization complete!

Script execution completed.
